In [1]:
#prepare the data
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the CSV file into a Pandas dataframe
df = pd.read_csv('edited_data.csv')

# Split the data into training and validation sets
train_data, val_data = train_test_split(df, test_size=0.2, random_state=42)


In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load the tokenizer for the transformer model
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

# Load the pre-trained model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased')


c:\Users\VICTUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\VICTUS\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequence

In [10]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [11]:
# Load the dataset
df = pd.read_csv('edited_data.csv')
max_length = max([len(verse.split()) for verse in df['Shloka']])
num_classes = df['Chapter No'].nunique()
X = df['Shloka']
y = pd.get_dummies(df['Chapter No']).values

In [12]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenize the input data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
vocab_size = len(tokenizer.word_index) + 1
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)


In [13]:
# Pad sequences to the same length
X_train_seq_padded = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_test_seq_padded = pad_sequences(X_test_seq, maxlen=max_length, padding='post')

# Define the model architecture
embedding_dim = 300
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(units=num_classes, activation='softmax'))

In [14]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_seq_padded, y_train, validation_data=(X_test_seq_padded, y_test), epochs=20, batch_size=64)


Epoch 1/20
9/9 [==============================] - 4s 140ms/step - loss: 2.8678 - accuracy: 0.0911 - val_loss: 2.8265 - val_accuracy: 0.1286
Epoch 2/20
9/9 [==============================] - 1s 90ms/step - loss: 2.8305 - accuracy: 0.1071 - val_loss: 2.8287 - val_accuracy: 0.1286
Epoch 3/20
9/9 [==============================] - 1s 90ms/step - loss: 2.8164 - accuracy: 0.1054 - val_loss: 2.8174 - val_accuracy: 0.1571
Epoch 4/20
9/9 [==============================] - 1s 91ms/step - loss: 2.8081 - accuracy: 0.1125 - val_loss: 2.8222 - val_accuracy: 0.1286
Epoch 5/20
9/9 [==============================] - 1s 91ms/step - loss: 2.7005 - accuracy: 0.1429 - val_loss: 2.7801 - val_accuracy: 0.0929
Epoch 6/20
9/9 [==============================] - 1s 93ms/step - loss: 2.3550 - accuracy: 0.1464 - val_loss: 3.2770 - val_accuracy: 0.0714
Epoch 7/20
9/9 [==============================] - 1s 86ms/step - loss: 2.2540 - accuracy: 0.1786 - val_loss: 3.4053 - val_accuracy: 0.0857
Epoch 8/20
9/9 [==========

In [15]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test_seq_padded, y_test)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

5/5 [==============================] - 0s 10ms/step - loss: 5.8900 - accuracy: 0.1286
Test loss: 5.889999866485596
Test accuracy: 0.12857143580913544


In [20]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from gensim.models import KeyedVectors

# Load the dataset
df = pd.read_csv('edited_data.csv')
max_length = max([len(verse.split()) for verse in df['Shloka']])
num_classes = df['Chapter No'].nunique()
X = df['Shloka']
y = pd.get_dummies(df['Chapter No']).values



In [24]:
# Load the embeddings
word_vectors = {}
with open('glove.6B.300d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        word_vectors[word] = vector

In [25]:
# Tokenize the input data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
vocab_size = len(tokenizer.word_index) + 1
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [26]:
# Pad sequences to the same length
X_train_seq_padded = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_test_seq_padded = pad_sequences(X_test_seq, maxlen=max_length, padding='post')

# Create the embedding matrix
embedding_dim = 300
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    if word in word_vectors:
        embedding_matrix[i] = word_vectors[word]

In [27]:
# Define the model architecture
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length, weights=[embedding_matrix], trainable=False))
model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(units=num_classes, activation='softmax'))

In [28]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [29]:
# Train the model
model.fit(X_train_seq_padded, y_train, validation_data=(X_test_seq_padded, y_test), epochs=30, batch_size=64)

Epoch 1/30
9/9 [==============================] - 4s 89ms/step - loss: 2.8865 - accuracy: 0.0911 - val_loss: 2.8732 - val_accuracy: 0.1286
Epoch 2/30
9/9 [==============================] - 0s 53ms/step - loss: 2.8552 - accuracy: 0.1071 - val_loss: 2.8124 - val_accuracy: 0.1286
Epoch 3/30
9/9 [==============================] - 0s 53ms/step - loss: 2.8189 - accuracy: 0.0893 - val_loss: 2.8245 - val_accuracy: 0.1286
Epoch 4/30
9/9 [==============================] - 0s 53ms/step - loss: 2.8170 - accuracy: 0.1232 - val_loss: 2.8322 - val_accuracy: 0.1286
Epoch 5/30
9/9 [==============================] - 0s 51ms/step - loss: 2.8193 - accuracy: 0.1071 - val_loss: 2.8250 - val_accuracy: 0.1286
Epoch 6/30
9/9 [==============================] - 0s 49ms/step - loss: 2.8175 - accuracy: 0.1071 - val_loss: 2.8192 - val_accuracy: 0.1286
Epoch 7/30
9/9 [==============================] - 0s 50ms/step - loss: 2.8189 - accuracy: 0.1089 - val_loss: 2.8324 - val_accuracy: 0.1286
Epoch 8/30
9/9 [===========

In [30]:
loss, accuracy = model.evaluate(X_test_seq_padded, y_test)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

5/5 [==============================] - 0s 10ms/step - loss: 2.8606 - accuracy: 0.1286
Test loss: 2.860588312149048
Test accuracy: 0.12857143580913544


In [ ]:

# Load the embeddings
word_vectors = {}
with open('glove/glove.6B.300d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        word_vectors[word] = vector

# Tokenize the input data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
vocab_size = len(tokenizer.word_index) + 1
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences to the same length
X_train_seq_padded = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_test_seq_padded = pad_sequences(X_test_seq, maxlen=max_length, padding='post')

# Create the embedding matrix
embedding_dim = 300
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    if word in word_vectors:
        embedding_matrix[i] = word_vectors[word]

# Define the model architecture
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length, weights=[embedding_matrix], trainable=False))
model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(units=num_classes, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_seq_padded, y_train, validation_data=(X_test_seq_padded, y_test), epochs=20, batch_size=64)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test_seq_padded, y_test)
print('Test loss:', loss)
print('Test accuracy:', accuracy)


In [21]:
# Download pre-trained GloVe embeddings
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip -d glove

'wget' is not recognized as an internal or external command,
operable program or batch file.
'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [18]:
from gensim.models import KeyedVectors